# Sample From Mesh Files

In this notebook, point clouds are created from mesh files using poisson disk sampling.

In [6]:
%load_ext autoreload
%autoreload 2

import os

import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
def load_mesh_and_sample_points(file_name, num_points=5000):
    # Default number of points as indicated in the thesis
    mesh = o3d.io.read_triangle_mesh(file_name)
    pc = mesh.sample_points_poisson_disk(num_points)
    return np.asarray(pc.points)

In [ ]:
# Load data and convert to point clouds
path = 'mesh_files_new'
count = 0
for idx, file in tqdm(enumerate(os.listdir(path))):
    file_path = os.path.join(path, file)
    if os.path.isfile(file_path):
        label = ""
        for s in file[:-3]:
            if s.isalpha():
                label += s
        point_cloud = load_mesh_and_sample_points(file_path)
        np.savez('point_clouds/' + label + str(count) + '.npz', objects=point_cloud, classes=label)
        count += 1

In [ ]:
# This step was only needed once
# Can be ignored
path = 'point_clouds'
objects = []
labels = []
for idx, file in tqdm(enumerate(os.listdir(path))):
    file_path = os.path.join(path, file)
    if os.path.isfile(file_path):
        load_table = np.load(file_path, allow_pickle=True)
        if len(load_table['objects'].shape) > 2:
            for i in range(load_table['objects'].shape[0]):
                objects.append(load_table['objects'][i])
                labels.append(load_table['classes'][i])
        else:
            objects.append(load_table['objects'])
            labels.append(load_table['classes'])

In [ ]:
np.savez('point_clouds/all_point_clouds', objects=objects, labels=labels)

In [7]:
load_clouds = np.load('point_clouds/all_point_clouds.npz', allow_pickle=True)
point_cloud_data = [load_clouds['objects'], load_clouds['labels']]

In [ ]:
def create_scatterplot_3d(data, descriptor):
    figure = plt.figure(figsize=(5, 5))
    ax = figure.add_subplot(111, projection='3d')

    # Map classes to colors
    unique_classes = data['class'].unique()
    colors = plt.cm.tab10(range(len(unique_classes)))
    class_color_map = dict(zip(unique_classes, colors))

    # Plot points by class
    for class_label in unique_classes:
        class_data = data[data['class'] == class_label]
        ax.scatter(
            class_data['evrap_x'], class_data['evrap_y'], class_data['evrap_z'],
            color=class_color_map[class_label], label=class_label, s=50
        )

    ax.legend()
    ax.view_init(80, 10)
    ax.set_title(descriptor)


def create_scatterplot_2d(data, descriptor):
    figure = plt.figure(figsize=(5, 5))
    ax = figure.add_subplot(111)

    # Map classes to colors
    unique_classes = data['class'].unique()
    colors = plt.cm.tab10(range(len(unique_classes)))
    class_color_map = dict(zip(unique_classes, colors))

    # Plot points by class
    for class_label in unique_classes:
        class_data = data[data['class'] == class_label]
        ax.scatter(
            class_data['samp_x'], class_data['samp_y'],
            color=class_color_map[class_label], label=class_label, s=50
        )

    ax.legend()
    ax.set_title(descriptor)


def create_scatterplot_1d(data, descriptor):
    figure = plt.figure(figsize=(5, 5))
    ax = figure.add_subplot(111)

    # Map classes to colors
    unique_classes = data['class'].unique()
    colors = plt.cm.tab10(range(len(unique_classes)))
    class_color_map = dict(zip(unique_classes, colors))

    # Plot points by class
    for class_label in unique_classes:
        class_data = data[data['class'] == class_label]
        x = np.arange(len(class_data[descriptor]))
        ax.scatter(
            x, class_data[descriptor], color=class_color_map[class_label], label=class_label,
        )

    ax.legend()
    ax.set_title(descriptor)

## Runtime Overview

The descriptors used have the following complexities. 

- EVARP: $O(nm)$
- SAMP: $O(nm)$
- SCOMP: $O(n \cdot (m$ log $m)$
- SIRM: $O(nm)$
- Shell Model: $O(nm)$
- Sector Model: $O(nm)$
- Combined Model: $O(n(m + m))$
- FPFH: $O(nmk)$

Notation:
- n: Number of point clouds
- m: Number of points per point cloud
- k: A hyperparameter of FPFH (relatively small), similar to k in k-NN